aim problems may be designed for the following topics so that students can get hands on experience in using python for natural language processing .part of speech tagging N-GRAM

In [1]:
import nltk
from nltk import word_tokenize
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.


True

In [2]:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

In [3]:
sentence=word_tokenize("this is a sentence")
nltk.pos_tag(sentence)

[('this', 'DT'), ('is', 'VBZ'), ('a', 'DT'), ('sentence', 'NN')]

n grams in sentiment analysis

In [4]:
import numpy as np
import pandas as pd
from sklearn import model_selection,naive_bayes,svm

In [5]:
df=pd.read_csv("/content/sample_data/Reviews.csv",engine='python',error_bad_lines=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
Skipping line 23723: unexpected end of data


In [6]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [8]:
df.shape

(23721, 10)

In [9]:
df.isnull().sum()

Id                        0
ProductId                 0
UserId                    0
ProfileName               0
HelpfulnessNumerator      0
HelpfulnessDenominator    0
Score                     0
Time                      0
Summary                   0
Text                      0
dtype: int64

In [10]:
df['Score'].value_counts()

5    14643
4     3498
1     2149
3     2053
2     1378
Name: Score, dtype: int64

In [11]:
df['ratings']=np.where(df['Score']>=3,1,0)

In [13]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,ratings
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,1
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,0
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,1
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,0
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,1


In [14]:
df['ratings'].value_counts()

1    20194
0     3527
Name: ratings, dtype: int64

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
x_train,x_test,y_train,y_test=train_test_split(df['Text'],df['ratings'],random_state=50)

In [17]:
print(x_train)

1930     I threw this away after about a week of owners...
20599    It is hard for me to assign stars to the cat l...
860      The chocolate on the stick was all melted and ...
18913    So, I really really wanted to like this stuff,...
15464    It was nice to find an organic instant oatmeal...
                               ...                        
15649    My 2 year old German Shepherd loves this treat...
22637    So I have never tried Sencha before. Maybe it ...
10123    I have not had pork rinds in more than 20 year...
5600     Very good coffee.  Can't always find it in our...
14000    My Malamutes aren't too picky, except one who ...
Name: Text, Length: 17790, dtype: object


In [18]:
print(y_train)

1930     0
20599    1
860      1
18913    0
15464    1
        ..
15649    1
22637    0
10123    1
5600     1
14000    1
Name: ratings, Length: 17790, dtype: int64


In [19]:
from sklearn.feature_extraction.text import CountVectorizer

In [20]:
vect=CountVectorizer(min_df=5,ngram_range=(1,2),).fit(x_train)

In [21]:
len(vect.get_feature_names())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


46043

In [22]:
x_train_vectorized=vect.transform(x_train)

In [23]:
from sklearn.naive_bayes import MultinomialNB

In [25]:
model=MultinomialNB()
model.fit(x_train_vectorized,y_train)

MultinomialNB()

In [26]:
predictions=model.predict(vect.transform(x_test))

In [27]:
print(predictions)

[1 1 1 ... 1 1 1]


In [28]:
from sklearn.metrics import classification_report,confusion_matrix

In [29]:
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))

[[ 643  265]
 [ 324 4699]]
              precision    recall  f1-score   support

           0       0.66      0.71      0.69       908
           1       0.95      0.94      0.94      5023

    accuracy                           0.90      5931
   macro avg       0.81      0.82      0.81      5931
weighted avg       0.90      0.90      0.90      5931



In [30]:
from sklearn.metrics import roc_auc_score
print("AUC score is ",roc_auc_score(y_test,predictions))

AUC score is  0.8218232474230873
